## 객체 인식을 위한 신경망

- 객체 인식(Object Detection) : 이미지/영상 내 객체를 식별하는 기술

  - 여러 가지 객체에 대한 **분류**
  
  - 객체의 **위치 정보** 검출

- 1-stage detector

  - 분류와 위치 검출을 동시에 행하는 방법

  - 비교적 빠른 속도, 낮은 정확도

  - ex) YOLO

- 2-stage detector

  - 분류와 위치 검출을 순차적으로 행하는 방법

  - 비교적 느린 속도, 높은 정확도
  
  - ex) R-CNN

- 과거의 객체 인식 알고리즘들은 **sliding window** 방식

  - 일정 크기의 window로 이미지의 모든 영역을 탐색하며 객체를 검출하는 방식 -> 비효율적

- 현재는 선택적 탐색(selective search) 알고리즘을 적용한 **후보 영역(region proposal)** 방식을 많이 사용함

---

#### R-CNN

- 이미지 분류를 수행하는 `CNN`과

- 객체가 있을 법한 영역을 제안하는 `후보 영역(region proposal)` 알고리즘을 결합함

<img src="https://production-media.paperswithcode.com/methods/new_splash-method_NaA95zW.jpg" width="600px">

- (1) 이미지를 입력 받음
<br><br>
- (2) 후보 영역을 추출함

    - 선택적 탐색 알고리즘으로 **2000개의 bounding box**를 추출하고 **crop** 함

    - crop 된 이미지들을 같은 크기로 통일함
<br><br>
- (3) CNN 적용

    - 추출된 2000개의 이미지에 각각 CNN을 적용함
<br><br>
- (4) 분류

    - 각 이미지(영역)를 분류하여 전체 결과를 도출함

##### - 선택적 탐색

- 객체가 있을 만한 **후보 영역**을 알아내는 방법

- (1) 초기 영역 생성

  - 이미지를 다수의 영역으로 분할함
  
  - 하나의 객체가 하나의 영역에 할당될 수 있도록 최대한 많은 영역을 생성함
<br><br>
- (2) 작은 영역의 통합

  - 서로 비슷한 영역끼리 통합함

  - 탐욕 알고리즘을 사용하여 비슷한 영역이 하나로 통합될 때까지 반복됨
<br><br>
- (3) 후보 영역 생성

  - 통합된 영역을 기반으로 후보 영역(bounding box)를 추출함

- 이후, 추출된 후보 영역에 CNN을 적용하게 됨

##### - 공간 피라미드 풀링

- 후보 영역을 CNN에 적용하기 위해, 이미지를 `동일한 크기`로 고정해야 함
  
  -> 객체의 일부가 잘리거나 생김새가 변형될 수 있는 문제가 있음
<br><br>
- 이미지 크기에 관계없이 Conv Layer을 적용하고, FC Layer에 전달되기 전에 **동일한 크기로 Pooling**을 적용해주는 방법
  
  -> 이미지의 특징 보존 효과

##### - R-CNN 단점

- 위 3단계의 복잡한 학습 과정

- 오랜 학습 시간과 대용량 저장 공간 필요

- Object Detection 속도 문제

  -> Fast R-CNN 등장

---

#### Fast R-CNN

- R-CNN의 속도 문제를 개선하기 위해 `RoI(Region of Interest Proposal) Pooling` 도입

  - 크기가 다른 영역마다 **stride를 다르게** 하여 Max Pooling을 적용하는 방법
  
  - 공간 피라미드 풀링보다 빠르게 각 RoI의 크기를 동일하게 맞출 수 있음

---

#### Faseter R-CNN

- `후보 영역 추출` 과정을 `CNN 내부`에서 진행할 수 있도록 설계함

- Fast R-CNN에 후보 영역 추출 네트워크 `RPN(Region Proposal Network)` 추가

##### - 후보 영역 추출 네트워크

  - NxN 크기의 작은 window 영역의 이미지를 입력받고, 해당 영역에 객체의 존재 유무를 판단함

  - 이를 sliding window 방식으로 하여 모든 영역에 대해 판단함

  - 하지만, 고정된 NxN window만으로는 다양한 크기의 이미지를 처리하기 어려움
  
  - 따라서, 여러 크기와 비율의 `reference box` k개를 미리 정의하고, 각각의 sliding window 위치마다 박스 k개를 출력하도록 설계함

<img src="https://blog.kakaocdn.net/dn/DQelq/btqK7cbAhww/o9FmiN4n8ilBqdQ2AJ4Zck/img.png" width="450px">

- NxN window 크기로 이미지를 sliding 하며, 전체 영역에 대해 객체 검출을 함

- 이 때, 각 window 영역에 대해 k개의 reference box가 생성됨

  - 각 window에 대해서는 다음과 같은 출력값이 나옴

    - 객체 유무 분류 : 객체/배경 -> 2개

    - 바운딩 박스 좌표 : (x, y, w, h) -> 4개

  - 따라서, 각 window에 해당하는 k개의 reference box에 대해서는 다음과 같은 출력값이 나옴

    - 객체 유무 분류 : 2k개
    
    - 바운딩 박스 좌표 : 4k개